In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os
import pickle
import datetime

In [2]:
SCOPES = [
    "https://www.googleapis.com/auth/tasks",
    "https://www.googleapis.com/auth/calendar.readonly",
]


def authenticate_google_services():
    creds = None
    # Check if token.pickle already exists (it contains the access and refresh tokens)
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(
                port=8080,
                open_browser=True,
                authorization_prompt_message="Please visit this URL: {url}",
                success_message="Authentication complete. You may close this tab.",
                no_browser=False,
                enable_local_termination=False,
            )
        # Save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return creds


def get_services():
    creds = authenticate_google_services()

    # Create service for Google Tasks
    tasks_service = build("tasks", "v1", credentials=creds)

    # Create service for Google Calendar
    calendar_service = build("calendar", "v3", credentials=creds)

    return tasks_service, calendar_service


def get_calendar_events(calendar_service):
    # Get the current time in ISO format
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time

    # Retrieve events from the primary calendar
    events_result = (
        calendar_service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )

    events = events_result.get("items", [])

    return events


def get_cal(calendar_service):
    page_token = None
    while True:
        calendar_list = calendar_service.calendarList().list(pageToken=page_token).execute()
        for calendar_list_entry in calendar_list['items']:
            print(calendar_list_entry['summary'])
            print(calendar_list_entry['id'])
        page_token = calendar_list.get('nextPageToken')
        if not page_token:
            break
    page_token = None
    while True:
        events = calendar_service.events().list(calendarId='drm77o78hek96lja8b5ujiaigo@group.calendar.google.com', pageToken=page_token).execute()
        for event in events['items']:
            # print(event)
            if event['status'] == 'confirmed':
                print(event['summary'])
        page_token = events.get('nextPageToken')
        if not page_token:
            break
    return 



def get_tasks(tasks_service):
    # Get all the task lists available in your account
    tasklists_result = tasks_service.tasklists().list().execute()
    return tasklists_result.get("items", [])


In [4]:
tasks_service, calendar_service = get_services()

In [9]:
page_token = None
while True:
    events = calendar_service.events().list(calendarId='drm77o78hek96lja8b5ujiaigo@group.calendar.google.com', pageToken=page_token).execute()
    i = 0
    for event in events['items']:
        # print(event)
        if event['status'] == 'confirmed':
            print(event['summary'])
            print(event['start'])
            print(event['end'])
        i += 1
        if i == 5:
            break
    page_token = events.get('nextPageToken')
    if not page_token:
        break

Database Systems
{'dateTime': '2022-06-23T08:10:00-04:00', 'timeZone': 'America/New_York'}
{'dateTime': '2022-06-23T10:00:00-04:00', 'timeZone': 'America/New_York'}
Dinner <3
{'dateTime': '2022-07-01T19:00:00-04:00', 'timeZone': 'America/New_York'}
{'dateTime': '2022-07-01T21:00:00-04:00', 'timeZone': 'America/New_York'}
Lunch
{'dateTime': '2022-06-27T12:00:00-04:00', 'timeZone': 'America/New_York'}
{'dateTime': '2022-06-27T13:15:00-04:00', 'timeZone': 'America/New_York'}
Denaly Birthday
{'dateTime': '2022-07-12T19:00:00-04:00', 'timeZone': 'America/New_York'}
{'dateTime': '2022-07-12T22:00:00-04:00', 'timeZone': 'America/New_York'}
Fishing Moment
{'dateTime': '2022-07-19T18:30:00-04:00', 'timeZone': 'America/New_York'}
{'dateTime': '2022-07-19T20:30:00-04:00', 'timeZone': 'America/New_York'}
Tchaikovsky
{'dateTime': '2023-03-11T17:15:00-05:00', 'timeZone': 'America/New_York'}
{'dateTime': '2023-03-11T18:30:00-05:00', 'timeZone': 'America/New_York'}
Fix CK Pure State Reconstruction
{'d